## Setup
Uncomment the below cells to install objectbox

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import ObjectBox
from langchain_text_splitters import CharacterTextSplitter

In [3]:
# splitting txt file to insert into objectbox

documents = TextLoader("../../modules/state_of_the_union.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

## Creating an ObjectBox instance

In [4]:
ob = ObjectBox.from_documents(documents=docs, embedding=HuggingFaceEmbeddings(), embedding_dimensions=768)

## Similarity Search

In [5]:
query = "What did the president say about Ketanji Brown Jackson"
similarity_results = ob.similarity_search(query)

In [6]:
print(similarity_results[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## Similarity search with score
The returned distance score is Euclidean distance. Therefore, a lower score is better.

In [7]:
similarity_results_with_score = ob.similarity_search_with_score(query, 1)

In [8]:
print(similarity_results_with_score[0][0].page_content)
print("score: ", similarity_results_with_score[0][1])

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
score:  1.2032089233398438


## ObjectBox as a Retreiver

ObjectBox, as all the other vector stores, is a LangChain Retriever. 

In [9]:
retriever = ob.as_retriever()
retriever

VectorStoreRetriever(tags=['ObjectBox', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.objectbox.ObjectBox object at 0x17af92530>)

We right now support only similarity as search_type and not mmr and similarity_score_threshold

In [10]:
retriever = ob.as_retriever(search_type="similarity")
retriever

VectorStoreRetriever(tags=['ObjectBox', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.objectbox.ObjectBox object at 0x17af92530>)

In [11]:
query = "What did the president say about Ketanji Brown Jackson"
retriever.get_relevant_documents(query)[0]

Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'id': 32})